# **Lab1. Сравнение предложений**

Okapova Akerke

15BD02047

okapova.akerke@gmail.com

http://github.com/akerukesha

Задача 1: сравнение предложений

Дан набор предложений, скопированных из Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:
- кошки (животные)
- UNIX-утилита cat для вывода содержимого файлов
- версии операционной системы OS X, названные в честь семейства кошачьих
Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

Выполните следующие шаги:
1. Скачайте файл с предложениями (sentences.txt).
2. Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().
3. Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.
4. Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.
5. Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.
6. Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.
7. Запишите полученные числа в ответ, разделив пробелом.
8. Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?

In [6]:
import numpy as np
import re
from scipy.spatial.distance import cosine

In [12]:
def get_sents_and_words(file):
    ''' возвращает список предложений и множество слов '''
    result_sents, result_words = [], []

    with open(file, 'r') as f:
        raw_sents = f.readlines()
    print("RAW_SENTENCES")
    print(raw_sents)
    for sent in raw_sents:
        sent = sent.lower()
        split_words = [x for x in re.split('[^a-z]', sent) if x != '']
        result_sents.append(split_words)
        result_words.extend(split_words)

    result_words = set(result_words)
    result_words = dict(zip([x for x in range(len(result_words))], result_words))
    return result_sents, result_words

LINES, WORDS = get_sents_and_words("sentences.txt")
print("TOKENS")
print(LINES)
print("WORDS")
print(WORDS)

RAW_SENTENCES
['In comparison to dogs, cats have not undergone major changes during the domestication process.\n', 'As cat simply catenates streams of bytes, it can be also used to concatenate binary files, where it will just concatenate sequence of bytes.\n', 'A common interactive use of cat for a single file is to output the content of a file to standard output.\n', 'Cats can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small animals.\n', 'In one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.\n', 'The domesticated cat and its closest wild ancestor are both diploid organisms that possess 38 chromosomes and roughly 20,000 genes.\n', 'Domestic cats are similar in size to the other members of the genus Felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).\n', 'However, if the output is piped or redirected, cat is unnecessary.\n', 'cat with one named file is saf

In [13]:
def init_matrix(sents, words):
    ''' инициализация матрицы '''
    result = np.zeros((len(sents), len(words)))

    for i in range(len(sents)):
        for j in range(len(words)):
            result[i, j] = word_count(words[j], sents[i])

    return result


def word_count(word, sent):
    ''' число вхождений слова в предложение '''
    cnt = 0
    for w in sent:
        if w == word:
            cnt = cnt + 1
    return cnt

MATRIX = init_matrix(LINES, WORDS)
print(MATRIX)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
def get_distincts(matrix, first):
    ''' косинусное расстояние предложения до всех других '''
    result = {}
    
    for i in range(0, first):
        result[i] = cosine(matrix[first,], matrix[i])
    
    for i in range(first + 1, len(matrix)):
        result[i] = cosine(matrix[first,], matrix[i])

    return result  

FINAL_DICT = get_distincts(MATRIX, 0)
FINAL_SORT = sorted(FINAL_DICT.items(), key=lambda x: x[1])
print(FINAL_SORT[0][0], FINAL_SORT[1][0])

6 4
